<a href="https://colab.research.google.com/github/AsafShiloah/LLM_Biase_Manipulation/blob/main/lm_harness_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall datasets
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git@big-refactor
!pip install llama-cpp-python
!pip install huggingface_hub
!pip install llama-cpp-python
!pip install langchain
!pip install langchain-community
!pip install langchain llama-cpp-python
!pip install xlsxwriter
!pip install openpyxl
!pip install lm-eval
!pip install bitsandbytes

Found existing installation: datasets 3.0.1
Uninstalling datasets-3.0.1:
  Would remove:
    /usr/local/bin/datasets-cli
    /usr/local/lib/python3.10/dist-packages/datasets-3.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/datasets/*
Proceed (Y/n)? Y
  Successfully uninstalled datasets-3.0.1
  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git (to revision big-refactor) to /tmp/pip-req-build-gdnqrw4e
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-gdnqrw4e
  Running command git checkout -b big-refactor --track origin/big-refactor
  Switched to a new branch 'big-refactor'
  Branch 'big-refactor' set up to track remote branch 'big-refactor' from 'origin'.
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit 967eb4fa90b80ba4e8cc7a2fd171f44f0e384808
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (py

In [2]:
from huggingface_hub import notebook_login
from openpyxl import load_workbook
from openpyxl.formatting.rule import CellIsRule
from openpyxl.styles import PatternFill
import pandas as pd
import os
import evaluate
from langchain import LLMChain, PromptTemplate
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.llms import LlamaCpp
from langchain.schema import SystemMessage, HumanMessage
!rm -rf /content/LLM-Research-Technion
!git clone https://github.com/yotamgardosh/LLM-Research-Technion.git
os.chdir('LLM-Research-Technion')
from models_class import Model
from huggingface_hub import notebook_login

notebook_login()

Cloning into 'LLM-Research-Technion'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 89 (delta 35), reused 35 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 8.95 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [3]:
mistral_7b = Model("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF","Mistral_7b","openhermes-2.5-mistral-7b.Q4_K_M.gguf")
orca_2_7b = Model("TheBloke/Orca-2-7B-GGUF","Orca2_7b","orca-2-7b.Q4_K_M.gguf")
llama_2_7b = Model("TheBloke/Llama-2-7B-GGUF","Llama2_7b","llama-2-7b.Q4_K_M.gguf")

------------------------------------------------------------------------
Initialized model 'Mistral_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
------------------------------------------------------------------------



------------------------------------------------------------------------
Initialized model 'Orca2_7b' with the following settings:
Context length: 16000
Threads: 32
GPU layers: 0
Max tokens: 30
Temperature: 0.2
Top-p sampling: 0.6

Available Methods:
  - create_model_path(): Download model files to a specified path.
  - create_llm(): Create and configure a language model for use.
  - __str__(): Return a string representation of the model's settings.
--------------

In [4]:

mistral_llm = mistral_7b.create_llm()
orca_llm = orca_2_7b.create_llm
llama_llm = llama_2_7b.create_llm()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--OpenHermes-2.5-Mistral-7B-GGUF/snapshots/5682e25bb033d9d21f6d159859e21df4552c1f26/openhermes-2.5-mistral-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_mod

In [5]:
file_path = '/content/sys_eng_bench.csv'
syseng_df = pd.read_csv(file_path)

In [8]:
YAML_SysEngBench_string = '''
task: demo_sysengbench_zero_shot
dataset_path: rabell/SysEngBench
description: "The following are multiple-choice questions related to System Engineering."
test_split: test
output_type: multiple_choice
doc_to_text: |
  Question: {{question.strip()}}
  A. {{choiceA}}
  B. {{choiceB}}
  C. {{choiceC}}
  D. {{choiceD}}
  Answer:
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
'''

# Save the YAML configuration to a file
with open('/content/mmlu_SysEngBench.yaml', 'w') as f:
    f.write(YAML_SysEngBench_string)



In [14]:
!lm_eval \
    --model hf \
    --model_args pretrained=TheBloke/OpenHermes-2.5-Mistral-7B-GGUF  \
    --include_path ./ \
    --tasks demo_sysengbench_zero_shot \
    --output output/sysengbench_results.json


2024-09-26:11:38:08,925 INFO     [utils.py:145] Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-09-26:11:38:08,925 INFO     [utils.py:148] Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-09-26:11:38:08,925 INFO     [utils.py:161] NumExpr defaulting to 16 threads.
2024-09-26:11:38:09,070 INFO     [config.py:54] PyTorch version 2.4.0+cpu available.
2024-09-26:11:38:09,071 INFO     [config.py:101] TensorFlow version 2.17.0 available.
2024-09-26:11:38:12,495 INFO     [__init__.py:131] Using bundled libtpu.so (/usr/local/lib/python3.10/dist-packages/torch_xla/lib/libtpu.so)
Traceback (most recent call last):
  File "/usr/local/bin/lm_eval", line 5, in <module>
    from lm_eval.__main__ import cli_evaluate
  File "/usr/local/lib/python3.10/dist-packages/lm_eval/__init__.py", line 1, in <module>
    from .evaluator import evaluate, simple_evaluate
  File "/usr/local/

In [13]:
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks hellaswag \
    --limit 10



Traceback (most recent call last):
  File "/usr/local/bin/lm_eval", line 5, in <module>
    from lm_eval.__main__ import cli_evaluate
  File "/usr/local/lib/python3.10/dist-packages/lm_eval/__init__.py", line 1, in <module>
    from .evaluator import evaluate, simple_evaluate
  File "/usr/local/lib/python3.10/dist-packages/lm_eval/evaluator.py", line 12, in <module>
    import lm_eval.tasks
  File "/usr/local/lib/python3.10/dist-packages/lm_eval/tasks/__init__.py", line 7, in <module>
    from lm_eval.api.task import TaskConfig, Task, ConfigurableTask
  File "/usr/local/lib/python3.10/dist-packages/lm_eval/api/task.py", line 8, in <module>
    import evaluate
  File "/usr/local/lib/python3.10/dist-packages/evaluate/__init__.py", line 29, in <module>
    from .evaluation_suite import EvaluationSuite
  File "/usr/local/lib/python3.10/dist-packages/evaluate/evaluation_suite/__init__.py", line 7, in <module>
    from datasets import Dataset, DownloadConfig, DownloadMode, load_dataset
  Fil